In [ ]:
import sys
import os
sys.path.insert(0, os.path.abspath('../..'))
from experiments.iter1.util.importer_TCN import *
GRAPH_PATH = 'graph_saves/ex24_graphs/'
MODEL_PATH = 'model_saves/ex24_model'
uk_path = "../../src/data_preprocess/dataset/UKDATA_cleaned.csv"

In [ ]:
assert time_horizon > 0, "Time horizon must be a positive integer"
    
df = pd.read_csv(uk_path)

cleaner = TempCleaner(clean_pow_low, clean_in_low, clean_in_high, clean_out_low, clean_out_high, clean_delta_temp)
splitter = StdSplitter(train_days, val_days, test_days)
    
model = TCN(hidden_size, num_layers, input_size, time_horizon, dropout, seq_len)
trainer = TrainerWrapper(L.Trainer, 
                         max_epochs=num_epochs, 
                         callbacks=[EarlyStopping(monitor='val_loss', min_delta=0.0, patience=3, verbose=False, mode='min', strict=True)], 
                         gradient_clip_val=gradient_clipping)
optimizer = OptimizerWrapper(optim.Adam, model, lr=learning_rate)

model = MonteCarloPipeline.Builder() \
    .add_data(df) \
    .set_cleaner(cleaner) \
    .set_normalizer_class(MinMaxNormalizer) \
    .set_splitter(splitter) \
    .set_sequencer_class(AllTimeSequencer) \
    .set_target_column(TARGET_COLUMN) \
    .set_model(model) \
    .set_optimizer(optimizer) \
    .set_batch_size(batch_size) \
    .set_seq_len(seq_len) \
    .set_worker_num(NUM_WORKERS) \
    .set_error(NRMSE) \
    .set_train_error(RMSE) \
    .set_trainer(trainer) \
    .set_tuner_class(StdTunerWrapper) \
    .set_inference_samples(inference_samples) \
    .set_inference_dropout(inference_dropout) \
    .build()

model = EnsemblePipeline.Builder() \
        .set_pipeline(model) \
        .set_num_ensembles(num_ensembles) \
        .set_horizon_len(time_horizon) \
        .build()

model.fit()
os.mkdir(GRAPH_PATH)
ensemple_val_loss_arr = model.get_validation_loss()
print("VALIDATION LOSS GRAPH")
for loss_arr in ensemple_val_loss_arr:
    plot_loss(loss_arr, GRAPH_PATH + "val")
ensemple_train_loss_arr = model.get_training_loss()
print("TRAINING LOSS GRAPH")
for loss_arr in ensemple_train_loss_arr:
    plot_loss(loss_arr, GRAPH_PATH + "train")
    
model.save(MODEL_PATH)

In [ ]:
from experiments.iter1.util.evaluate import evaluate_model

df = pd.read_csv(uk_path)

cleaner = TempCleaner(clean_pow_low, clean_in_low, clean_in_high, clean_out_low, clean_out_high, clean_delta_temp)
splitter = StdSplitter(train_days, val_days, test_days)
    
model = TCN(hidden_size, num_layers, input_size, time_horizon, dropout, seq_len)
trainer = TrainerWrapper(L.Trainer, 
                         max_epochs=num_epochs, 
                         callbacks=[EarlyStopping(monitor='val_loss', min_delta=0.0, patience=3, verbose=False, mode='min', strict=True)], 
                         gradient_clip_val=gradient_clipping)
optimizer = OptimizerWrapper(optim.Adam, model, lr=learning_rate)

model = MonteCarloPipeline.Builder() \
    .add_data(df) \
    .set_cleaner(cleaner) \
    .set_normalizer_class(MinMaxNormalizer) \
    .set_splitter(splitter) \
    .set_sequencer_class(AllTimeSequencer) \
    .set_target_column(TARGET_COLUMN) \
    .set_model(model) \
    .set_optimizer(optimizer) \
    .set_batch_size(batch_size) \
    .set_seq_len(seq_len) \
    .set_worker_num(NUM_WORKERS) \
    .set_error(NRMSE) \
    .set_train_error(RMSE) \
    .set_trainer(trainer) \
    .set_tuner_class(StdTunerWrapper) \
    .set_inference_samples(inference_samples) \
    .set_inference_dropout(inference_dropout) \
    .build()

model = EnsemblePipeline.Builder() \
        .set_pipeline(model) \
        .set_num_ensembles(num_ensembles) \
        .add_test_error(NRMSE) \
        .add_test_error(NMLSCV) \
        .add_test_error(NMCRPS) \
        .set_horizon_len(time_horizon) \
        .build()

model.load(MODEL_PATH)
model.test()

evaluate_model(model, df, splitter, cleaner, TIMESTAMP, POWER, on_limit_w, off_limit_w, consecutive_points, seq_len, time_horizon, TARGET_COLUMN, error, temp_boundary, 0.95)